In [1]:
import dropbox
import os
import numpy as np
import pandas as pd
from scipy.stats import t
import math
import collections
from PIL import Image
from dropbox import DropboxOAuth2FlowNoRedirect
import matplotlib.pyplot as plt
import cv2
import pypyodbc
import sys
import traceback
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import BatchNormalization, concatenate, Input, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical  #?
from keras.optimizers import Adam

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
cnxn = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=028-GUMUS1-SRV;"
                      "Database=KOTON_DB;"
                      "Trusted_Connection=yes;")

In [3]:
access_token='0DQLcsXIojcAAAAAAAgybeB3DOmXKK7bRTLyYwkthbrAiGmpQR4AuGINQkjBXhif'
dbx = dropbox.Dropbox(access_token)

retrieved_image_df = pd.read_excel('C:/Users/Recep/Koton/Data/retrieved_image_list.xlsx', sheet_name='Sheet1')

In [4]:
def data_set(codes, indexes):
    """Takes list of option codes as input and returns related dataset, response set (sales amount)
    and genuine option codes that included in e-commerce table.
    """
    
    if len(codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
    
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    
    codes_split = np.array_split(codes, numOfBatches)
    indexes_split = np.array_split(indexes, numOfBatches)
    
    # to prevent empty targets
    gen_codes = []
    gen_indexes = []
    target = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)

    for code, index in zip(codes, indexes):
        try:
            target.loc[code]
            gen_codes.append(code)
            gen_indexes.append(index)
        except:
            pass        
        
    
    string_inp = "SELECT * \
  FROM [KOTON_DB].[dbo].[Input_table] \
  WHERE "
    
#     numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    gen_codes_splited = np.array_split(gen_codes, numOfBatches)
    dataset = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag1 = True
        for i in gen_codes_splited[batch]:
            if flag1:
                _.append("OptionCode='"+i+"'")
                flag1 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query1 = string_inp + ''.join(_)
        dataset.append(pd.read_sql_query(query1, cnxn, index_col='optioncode'))
    
    dataset = pd.concat(dataset)
    
    
    return dataset, target, gen_codes, gen_indexes

In [13]:
def tarrain(numberSamples, startIndex, batchSize=64, return_data=False):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet', pooling='max')

    first = bn_model.output
    for layer in bn_model.layers:
        layer.trainable = False

    model = Dense(256, activation='relu')(first)
    model = Dropout(0.5)(model)
    model = Dense(10, activation='softmax')(model)

    model_ = Model(inputs=[bn_model.input], outputs=model)

    model_.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
#     dset = dset.apply(LabelEncoder().fit_transform)
    dset = pd.get_dummies(dset)
    tar = pd.get_dummies(tar)
    print(dset.shape)
    print(tar.shape)
       
    if batchSize>dset.shape[0]:
        batchSize = dset.shape[0]
        print('Batch size is automatically adjusted.')
    
    dset, dset_test, tar, tar_test = train_test_split(dset, tar, test_size=0.01, random_state=42)
    gen_codes, gen_codes_test, gen_indexes, gen_indexes_test = train_test_split(gen_codes, gen_indexes, 
                                                                                test_size=0.01, random_state=42)
    print(dset.shape)
    
    numOfBatches, lastBatch = divmod(dset.shape[0], batchSize-1)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    valid_pic1 =[]
    for code, picture_index in zip(gen_codes_test, gen_indexes_test):
        im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
        if (math.isnan(float(im_index))==False):
            filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
            path = '/Koton_Image_Files/'+filename
            try:
                md, res = dbx.files_download(path)
                temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                assert temp_pic.shape == (299, 299, 3)
                valid_pic1.append(temp_pic)
            except:
                errorLog.append(path+' Not found')
    valid_pic1 = preprocess_input(np.array(valid_pic1))
#     valid_bottleneck_features = bn_model.predict(valid_pic1)
    
    for batch in range(numOfBatches):
        pic1 =[]
        print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
        for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
            im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
            if (math.isnan(float(im_index))==False):
                filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                path = '/Koton_Image_Files/'+filename
                try:
                    md, res = dbx.files_download(path)
                    temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                    assert temp_pic.shape == (299, 299, 3)
                    pic1.append(temp_pic)
                except:
                    errorLog.append(path+' Not found')
                    batch += 1 #ignore defective batch
        pic1 = preprocess_input(np.array(pic1))
        try:
            model_.fit(pic1, tars[batch], validation_data=(valid_pic1, tar_test), epochs=60)
            if batch%3 ==0:
                timestr = time.strftime("%Y%m%d")
                model_.save('model'+timestr+'.h5')
        except:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            errorLog.append(''.join('!! ' + line for line in lines))
    if return_data:
        return model_, errorLog, dset, dset_test, tar, tar_test, gen_codes, gen_codes_test, gen_indexes, gen_indexes_test
    else:
        return model_, errorLog
                    

In [ ]:
model, errorLog, dset, dset_test, tar, tar_test = tarrain(40000, 1724, 64, return_data=True)

(6144, 193)
(6144, 10)
(6082, 193)
Batch: 1/96
Train on 64 samples, validate on 62 samples
Epoch 1/60
64/64 [==============================] - 101s 2s/step - loss: 8.5426 - acc: 0.1562 - val_loss: 7.2880 - val_acc: 0.0968
Epoch 2/60
64/64 [==============================] - 97s 2s/step - loss: 9.2375 - acc: 0.2656 - val_loss: 8.1729 - val_acc: 0.1290
Epoch 3/60
64/64 [==============================] - 98s 2s/step - loss: 9.5140 - acc: 0.1719 - val_loss: 6.7239 - val_acc: 0.1129
Epoch 4/60
64/64 [==============================] - 97s 2s/step - loss: 7.0327 - acc: 0.2344 - val_loss: 5.7679 - val_acc: 0.1129
Epoch 5/60
64/64 [==============================] - 98s 2s/step - loss: 6.7272 - acc: 0.1406 - val_loss: 5.8762 - val_acc: 0.0806
Epoch 6/60
64/64 [==============================] - 97s 2s/step - loss: 7.1608 - acc: 0.1562 - val_loss: 4.9213 - val_acc: 0.0968
Epoch 7/60
64/64 [==============================] - 98s 2s/step - loss: 4.8422 - acc: 0.3125 - val_loss: 3.4644 - val_acc: 0.241

64/64 [==============================] - 97s 2s/step - loss: 3.8523 - acc: 0.2500 - val_loss: 2.2693 - val_acc: 0.2097
Epoch 3/60
64/64 [==============================] - 97s 2s/step - loss: 2.2371 - acc: 0.2812 - val_loss: 2.1563 - val_acc: 0.2581
Epoch 4/60
64/64 [==============================] - 97s 2s/step - loss: 1.9422 - acc: 0.3438 - val_loss: 2.2302 - val_acc: 0.1774
Epoch 5/60
64/64 [==============================] - 98s 2s/step - loss: 2.0735 - acc: 0.2969 - val_loss: 2.2126 - val_acc: 0.1774
Epoch 6/60
64/64 [==============================] - 98s 2s/step - loss: 1.8536 - acc: 0.3906 - val_loss: 2.2160 - val_acc: 0.2258
Epoch 7/60
64/64 [==============================] - 98s 2s/step - loss: 1.9371 - acc: 0.4062 - val_loss: 2.3319 - val_acc: 0.1774
Epoch 8/60
64/64 [==============================] - 97s 2s/step - loss: 1.8908 - acc: 0.3438 - val_loss: 2.1806 - val_acc: 0.2581
Epoch 9/60
64/64 [==============================] - 98s 2s/step - loss: 1.6589 - acc: 0.4219 - val_lo

64/64 [==============================] - 96s 2s/step - loss: 2.2623 - acc: 0.2656 - val_loss: 2.3375 - val_acc: 0.1290
Epoch 5/60
64/64 [==============================] - 97s 2s/step - loss: 2.1261 - acc: 0.2188 - val_loss: 2.2226 - val_acc: 0.1935
Epoch 6/60
64/64 [==============================] - 99s 2s/step - loss: 1.9912 - acc: 0.2031 - val_loss: 2.2196 - val_acc: 0.1613
Epoch 7/60
64/64 [==============================] - 97s 2s/step - loss: 1.8629 - acc: 0.2969 - val_loss: 2.1962 - val_acc: 0.2419
Epoch 8/60
64/64 [==============================] - 97s 2s/step - loss: 1.9849 - acc: 0.2656 - val_loss: 2.3107 - val_acc: 0.1290
Epoch 9/60
64/64 [==============================] - 97s 2s/step - loss: 1.8429 - acc: 0.2812 - val_loss: 2.2774 - val_acc: 0.1774
Epoch 10/60
64/64 [==============================] - 97s 2s/step - loss: 1.8396 - acc: 0.3906 - val_loss: 2.2095 - val_acc: 0.1452
Epoch 11/60
64/64 [==============================] - 97s 2s/step - loss: 1.7448 - acc: 0.3281 - val_

Epoch 6/60
64/64 [==============================] - 107s 2s/step - loss: 1.8590 - acc: 0.3125 - val_loss: 2.2143 - val_acc: 0.3226
Epoch 7/60
64/64 [==============================] - 103s 2s/step - loss: 1.9980 - acc: 0.2969 - val_loss: 2.1661 - val_acc: 0.2419
Epoch 8/60
64/64 [==============================] - 104s 2s/step - loss: 1.8708 - acc: 0.2812 - val_loss: 2.2734 - val_acc: 0.2097
Epoch 9/60
64/64 [==============================] - 99s 2s/step - loss: 1.8444 - acc: 0.3906 - val_loss: 2.2001 - val_acc: 0.2742
Epoch 10/60
64/64 [==============================] - 100s 2s/step - loss: 1.5881 - acc: 0.3906 - val_loss: 2.3436 - val_acc: 0.2419
Epoch 11/60
64/64 [==============================] - 100s 2s/step - loss: 1.7073 - acc: 0.4062 - val_loss: 2.2662 - val_acc: 0.2419
Epoch 12/60
64/64 [==============================] - 101s 2s/step - loss: 1.5124 - acc: 0.4688 - val_loss: 2.1926 - val_acc: 0.2903
Epoch 13/60
64/64 [==============================] - 99s 2s/step - loss: 1.5245 -

64/64 [==============================] - 98s 2s/step - loss: 1.1721 - acc: 0.5312 - val_loss: 2.3640 - val_acc: 0.2258
Epoch 46/60
64/64 [==============================] - 97s 2s/step - loss: 0.8899 - acc: 0.6562 - val_loss: 2.3087 - val_acc: 0.2581
Epoch 47/60
64/64 [==============================] - 97s 2s/step - loss: 1.0558 - acc: 0.5781 - val_loss: 2.3016 - val_acc: 0.2742
Epoch 48/60
64/64 [==============================] - 97s 2s/step - loss: 1.0402 - acc: 0.5938 - val_loss: 2.2828 - val_acc: 0.1935
Epoch 49/60
64/64 [==============================] - 97s 2s/step - loss: 1.0483 - acc: 0.5156 - val_loss: 2.4251 - val_acc: 0.2903
Epoch 50/60
64/64 [==============================] - 98s 2s/step - loss: 1.0712 - acc: 0.5469 - val_loss: 2.4358 - val_acc: 0.2742
Epoch 51/60
64/64 [==============================] - 97s 2s/step - loss: 1.0353 - acc: 0.5625 - val_loss: 2.4980 - val_acc: 0.2258
Epoch 52/60
64/64 [==============================] - 98s 2s/step - loss: 1.0762 - acc: 0.6094 -